# 1. import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 필요한 모듈, 데이터

!pip install sentence_transformers
!pip install konlpy

import pandas as pd
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm

# 모델, okt 저장
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
okt = Okt()

# 2. data load

In [ ]:
# 데이터
df = pd.read_pickle('/content/drive/MyDrive/6조_파이널PJT/data/SBERT/split_by_doc.pkl')
df2 = pd.read_pickle('/content/drive/MyDrive/6조_파이널PJT/data/SBERT/SBERT_final.pkl')
# df2 = pd.read_pickle('/content/drive/MyDrive/6조_파이널PJT/data/SBERT/SBERT_final_1000.pkl')
test = pd.read_pickle('/content/drive/MyDrive/6조_파이널PJT/data/SBERT/SBERT_test.pkl')

with open('/content/drive/MyDrive/6조_파이널PJT/data/SBERT/sbert_stop_words.txt', 'r') as file:
    string = file.read()
    stop_words = string.split('\n')

# 3. 함수
- find_titles
- find_content

In [ ]:
# 질문 -> 제목, 소제목
# output : str, li (제목, 소제목 리스트)

def find_titles(data, stop_words, question):

  # 질문 -> 제목
  tokenized_doc = okt.pos(question) # 질문 토큰화
  tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if ((word[1] == 'Number') | (word[1] == 'Noun') | (word[1] == 'Alpha')) \
                              & (word[0] not in stop_words)])
  
  # 질문 토큰 embedding
  n_gram_range = (2, 3)
  try:
    count = CountVectorizer(ngram_range=n_gram_range, stop_words=stop_words).fit([tokenized_nouns]) 
    qas_candidates = count.get_feature_names_out()
    qas_embedding = model.encode(qas_candidates)
  except: 
    qas_candidates = np.array([tokenized_nouns], dtype = object)
    qas_embedding = model.encode(qas_candidates)
    
  # 질문 embedding, 단어들과 코사인 유사도 확인
  qas_doc_embedding = model.encode([question])
  qas_candidate_embeddings = model.encode(qas_candidates)

  top_n = 1 # 코사인 유사도가 제일 높은 단어 순으로 n개 추출
  distances = cosine_similarity(qas_doc_embedding, qas_candidate_embeddings)
  keywords = [qas_candidates[index] for index in distances.argsort()[0][-top_n:]]
  keyword = keywords[0] # top_n = 1인 경우

  # 키워드가 없는 경우
  if len(keyword) == 0:
    title = ''
    subs_li = ''
    print('찾는 문서가 없습니다.')
    return title, subs_li

  # 키워드 공백 기준으로 분리 (ex. 지미 카터 -> [지미, 카터])
  keys = keyword.split(' ')
  title_li = list(data['title'])
  same = []
  for i in range(len(title_li)):
    for j in range(len(keys)):
      if keys[j] not in title_li[i]:
        break
      elif j == (len(keys)-1): # 타이틀에 키워드가 모두 들어간 경우
        same.append([i,title_li[i]])

  if len(same) == 1: # 키워드가 들어간 타이틀이 1개인 경우
    title = same[0][1] # 해당 문서의 제목 반환

  else: # 제목 리스트에서 코사인 유사도 검사
    key_len = len(keyword.replace(' ',''))
    distances_li = []
    keywords_li = []

    # 키워드가 들어간 타이틀이 2개 이상인 경우
    # 해당 문서들을 title_data로 저장
    if len(same) > 1: 
      title_data = pd.DataFrame(columns = data.columns)
      for i in range(len(same)):
        title_data.loc[i] = data.loc[same[i][0]]

    # 키워드가 들어간 타이틀이 없는 경우
    # 글자 수가 일치하는 문서들을 title_data로 저장
    else: 
      title_data = data.loc[data['title_len']==key_len]
      title_data.reset_index(drop=True, inplace=True)

    # title_data 문서 안에서 코사인 유사도 계산
    title_li = list(title_data['title'])
    for i in tqdm(range(len(title_li))):
      distances = cosine_similarity(qas_embedding, title_data.loc[i,'embedding'])
      distances_li.append(distances)
      keywords_li.append(max(map(max, distances_li[i])))
      title_idx = keywords_li.index(max(keywords_li))
    title = title_li[title_idx] # 유사도 가장 높은 단어를 제목으로 선정

  # 제목 -> 소제목
  dt = data[data['title'] == title]
  subs = dt['sub_name'].values[0]
  subs_idx = dt['sub_idx'].values[0]

  subs_li = ['기본']
  for i in range(len(subs)-1): # 내용 있는 소제목만 출력
    if subs_idx[i+1] - subs_idx[i] != 1:
      subs_li.append(subs[i])
  if len(subs) > 0:
    subs_li.append(subs[len(subs)-1])

  return title, subs_li

In [ ]:
# 제목, 소제목 -> 내용
# output : str (소제목에 해당하는 내용)

def find_content(data, split_data, title, subtitle):
  dt = data[data['title'] == title]

  # 1) desc -> desc 출력
  if subtitle == '기본':
    return dt['desc'].values[0]

  # 2) 그 외 특정 목차
  # 해당 목차의 내용을 출력
  else:
    try:
      j = dt['sub_name'].values[0].index(subtitle)
      st = dt['sub_idx'].values[0][j] + 1
      try:
        ed = dt['sub_idx'].values[0][j+1]
      except:
        ed = -1
      output = ' '.join(split_data[title].split('\n')[st:ed])
      return output
    except:
      return '잘못된 입력입니다.'

# 4. 실행

In [ ]:
# 실행
question = input('질문해주세요 : ')

title, sub_li = find_titles(df2, stop_words, question)
print('title:', title)
print('소제목 리스트\n', ', '.join(sub_li))

sub = input('소제목 선택 : ')
output = find_content(df2, df, title, sub)
print(output)

질문해주세요 : 체첸공화국 어디에 있어?


  0%|          | 0/51 [00:00<?, ?it/s]

title: 알브레히트 뒤러
소제목 리스트
 기본, 삶과 창작 활동, 1506-1514년, 1518-1520년, 업적
소제목 선택 : 1506-1514년
1509년부터 뒤러는 뉘른베르크 시위원회에 의해서 임명된 미술가가 되었다. 사람들은 도시의 예술 계획을 입안하는데 뒤러가 상당부분 관여를 했다고 가정을 한다. 이 시기 동안 작은 크기의 동판화와 목판화 이외에 중요한 목판화 연작을 간행했다. 그 당시 뒤러는 드라이포인트 기법을 이용해서 판화를 제작하려는 시도를 했다. 그렇게 해서 〈성녀 베로니카〉(1510년), 〈고통을 겪는 구세주〉(1512년), 〈참회하는 성 히에로니무스〉(1512년)가 제작되었다. 이 시기부터 뒤러의 작품에서는 목판화가 압도적으로 우세하게 되었고, 뒤러가 그린 회화는 점점 드물게 보게 되었다. 1512년 아기 예수를 안고 있는 성모 마리아의 모습을 그린 유화가 있다. 작은 크기의 동판화 연작의 상당 부분이 이 해에 제작되었다. 그리고 뒤러는 막시밀리안 황제로부터 자신의 목판화와 동판화를 복제하는 것을 금지할 수 있는 특별한 권리를 얻었다. 뒤러는 여러 번 황제 막시밀리안 1세의 주문을 받아서 일을 했다. 늦어도 1510/1511년 이후부터는 관계가 이루어졌다. 뒤러의 친구인 빌리발트 피리크하이머(Willibald Pirckheimer)가 그 관계를 중재했다. 모든 작품은 간접적으로나마 황제의 결혼과 명성을 위한 것이었다. 또한 이 시기에 그의 유명한 동판화 〈기사, 죽음과 악마〉,〈서재의 성 히에로니무스〉,〈멜랑콜리아I〉이 제작되었다. 또한 〈파움가르트너 제단화〉도 이 시기에 제작된 작품이다. 그리고 어머니가 돌아가시기 2달 전에 뒤러는 목판화로 어머니의 초상화를 제작했는데, 이 그림은 서양 미술사에서 죽을 정도로 병약해진 인간을 그린 최초의 초상화였다.
